# Food Delivery Analysis

We have data from a food delivery app that contains several features related to food delivery from restaurants in various cities. The dataset can be found [here](https://machinehack.com/hackathon/visualization/visualization_challenge_1_analyze_and_visualize_the_food_delivery_time_for_different_cuisines/overview). In this notebook, we will preprocess the data and perform exploratory data analysis and visualization to obtain some meaningful insights into the data.

In [1]:
#! pip install plotly==5.5.0

In [2]:
import plotly
plotly.__version__

'5.5.0'

In [3]:
#required libraries
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import re

In [4]:
df = pd.read_csv("food_delivery.csv")
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_5192,"FTI College, Law College Road, Pune","Bakery, Fast Food, Rolls, Beverages",₹200,₹50,3.1,279,74,45 minutes
2,ID_6857,"FTI College, Law College Road, Pune","South Indian, North Indian",₹150,₹50,3.1,253,35,30 minutes
3,ID_3429,"FTI College, Law College Road, Pune","Biryani, Chinese",₹150,₹50,3.2,4,4,30 minutes
4,ID_4654,"FTI College, Law College Road, Pune","Burger, Fast Food",₹200,₹50,3.9,326,106,30 minutes


### Data Preprocessing

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9937 entries, 0 to 9936
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Restaurant     9937 non-null   object
 1   Location       9937 non-null   object
 2   Cuisines       9937 non-null   object
 3   Average_Cost   9937 non-null   object
 4   Minimum_Order  9937 non-null   object
 5   Rating         9937 non-null   object
 6   Votes          9937 non-null   object
 7   Reviews        9937 non-null   object
 8   Delivery_Time  9937 non-null   object
dtypes: object(9)
memory usage: 698.8+ KB


There are no null values in the dataset and all the columns are of object data type. `Restaurant`, `Location`, `Cuisines` are categorical whereas the rest of the columns are numerical.

*Let's see the different cuisines in our dataset.*

In [6]:
print("Different cuisines count: ",df.Cuisines.nunique())
df.Cuisines.value_counts()

Different cuisines count:  1967


North Indian                                   801
North Indian, Chinese                          556
Fast Food                                      417
South Indian                                   241
Chinese                                        229
                                              ... 
Cafe, Fast Food, Chinese                         1
Chinese, Thai, Seafood, North Indian, Momos      1
Chinese, Thai, Continental, North Indian         1
Chinese, Fast Food, Beverages, North Indian      1
Asian, Chinese, Sushi, Thai, Japanese, Poké      1
Name: Cuisines, Length: 1967, dtype: int64

Most of the restaurants offer multiple cuisines with `North Indian` and `Chinese` being popular ones.

In [ ]:
print("Number of Restaurants: ", df.Restaurant.nunique())

Number of Restaurants:  6736


In [ ]:
df.Delivery_Time.value_counts()

30 minutes     6641
45 minutes     2394
65 minutes      819
120 minutes      59
80 minutes       14
20 minutes        6
10 minutes        4
Name: Delivery_Time, dtype: int64

On average the longest time it takes to deliver food is around 2 hours and the shortest time is 10 minutes.

*Further we need to convert the numerical variables of object data type to numeric data type.*

In [7]:
#delivery time
t = df.Delivery_Time
y=[]
for i in range(0, len(t)):
 y.append(int(re.findall('[0-9]+', t[i])[0]))
df["deliveryTime"] = y

In [8]:
#convert rating to float data type
df['numRating']=df.Rating.str.replace(r'[^0-9.]+','0')
df['numRating']=df.numRating.astype('float').round(2)

#convert votes and reviews to integer data type
df["Votes"] = df.Votes.replace("-","0").astype('int')
df["Reviews"] = df.Reviews.replace("-","0").astype('int')

In [9]:
#average and minimum order value
df['avgCost'] = df.Average_Cost.str.replace('[^0-9]','')
df['minOrder'] = df.Minimum_Order.str.replace('[^0-9]','')

*Let's now look at the summary statistics of the numerical columns.*

In [10]:
#summary stats
df.describe()

,Votes,Reviews,deliveryTime,numRating
count,9937.000000,9937.000000,9937.000000,9937.000000
mean,201.042769,98.393479,37.089162,2.974872
std,498.393173,274.736765,12.520631,1.428590
min,0.000000,0.000000,10.000000,0.000000
25%,7.000000,2.000000,30.000000,3.000000
50%,38.000000,14.000000,30.000000,3.500000
75%,161.000000,66.000000,45.000000,3.800000
max,7661.000000,4966.000000,120.000000,4.900000


`Votes` and `Reviews` column seems to have potential outliers, as we can observe the huge difference in the mean and median values for these features. The average number of votes is almost double of the reviews.

In [11]:
#number of unique locations
df.Location.nunique()

32

In [12]:
df.Location.unique()[:2]

array(['FTI College, Law College Road, Pune', 'Mumbai Central'],
      dtype=object)

In [13]:
city_list = ['banglore','delhi','gurgaon', 'hyderabad', 'kolkata','mumbai','noida','pune']
city_dict = {}
unknown_loc = []

for loc in df.Location.unique():
  #split into individual words
  t = re.sub(',',' ',loc).split(' ')
  found=False
  for word in t:
    if word.lower() in city_list:
      city_dict[loc] = word
      found=True
      break
  if not found:
    city_dict[loc] = "unknown"
    unknown_loc.append(loc)

#locations where city is not from the list
unknown_loc

['Mico Layout, Stage 2, BTM Layout,Bangalore',
 'Laxman Vihar Industrial Area, Sector 3A, Gurgoan',
 'BTM Layout 1, Electronic City']

We can observe that these three locations have no city name and different spellings, so we'll correct these values and create a new city column using the `city_dict` dictionary.

In [14]:
#extracting cities from location
city_dict[unknown_loc[0]] = 'Banglore'
city_dict[unknown_loc[1]] = 'Gurgaon'
city_dict[unknown_loc[2]] = 'Banglore'
df['city'] = df.Location.map(city_dict)
df.city.unique()

array(['Pune', 'Mumbai', 'Noida', 'Delhi', 'Kolkata', 'Banglore',
       'Gurgaon', 'Hyderabad'], dtype=object)

In [15]:
#splitting multiple cuisines into separate rows
df['Cuisines']=df.Cuisines.str.replace(', ',',') 
df1=df.assign(cuisine=df.Cuisines.str.split(',')).explode('cuisine')
df1.cuisine.value_counts()

North Indian    4299
Chinese         3079
Fast Food       2831
Beverages       1232
Desserts        1069
                ... 
Hot dogs           2
Sri Lankan         1
Bohri              1
Greek              1
African            1
Name: cuisine, Length: 100, dtype: int64

The most popular among the cuisines include `North Indian`, `Chinese`, `Fast Food`, `Beverages` and `Desserts`. There are total 100 different cuisines available in the dataset.

In [16]:
#dropping redundant features
df1 = df1.drop(columns=['Cuisines','Average_Cost','Minimum_Order','Rating','Delivery_Time'])
df1.columns

Index(['Restaurant', 'Location', 'Votes', 'Reviews', 'deliveryTime',
       'numRating', 'avgCost', 'minOrder', 'city', 'cuisine'],
      dtype='object')

### Exploratory Data Analysis & Data Visualization

In [108]:
#default template and color for different cities
template = 'plotly'
cityColor = {}
bgcolor = ['rgb(124, 235, 163)','rgb(85, 188, 194)','rgb(122, 158, 230)','rgb(192, 198, 240)', 'rgb(154, 114, 207)',
           'rgb(218, 161, 237)', 'rgb(235, 164, 225)', 'rgb(222, 133, 175)']
linecolor = ['rgb(58, 153, 91)','rgb(32, 93, 97)','rgb(44, 79, 148)','rgb(20, 46, 224)', 'rgb(79, 32, 140)',
             'rgb(86, 30, 105)', 'rgb(145, 31, 130)', 'rgb(181, 27, 99)']
i=0
for city in df1.city.unique():
  cityColor[city] = [bgcolor[i], linecolor[i]]
  i+=1

First we'll look at the number of restaurants in each location and their average rating,

In [17]:
loc_wise_ratings = df1.groupby("Location")["numRating"].agg(["mean","count"])
loc_wise_ratings['mean'] = loc_wise_ratings['mean'].round(2)
loc_wise_ratings[:5]

,mean,count
Location,,
"BTM Layout 1, Electronic City",3.96,64
"Babarpur, New Delhi, Delhi",2.66,550
"Chandni Chowk, Kolkata",3.24,299
"Chatta Bazaar, Malakpet, Hyderabad",3.00,117
"D-Block, Sector 63, Noida",3.01,1797


In [127]:
fig = px.bar(loc_wise_ratings, x= 'mean', y=loc_wise_ratings.index, color = "count", labels={'count':'Number of Restaurants','mean':'Average Ratings'}, 
             orientation='h', height = 1000, width = 700, template=template)
fig.update_layout(title = "<b>Location wise Average Ratings</b>")
fig.show()

The Location wise average ratings plot shows the average of the ratings taken from the number of restaurants. `BTM Layout 1, Electronic City` has the highest average rating of 3.9 stars with 64 restaurants in it.

In [102]:
restaurants = df1.groupby('city')['Restaurant'].agg('count')
title="<b>Total Number of Restaurants</b>"
fig = px.pie(values=restaurants, names=restaurants.index, title=title, labels={'value':'Number of Restaurants'}, width=400, height=400, template=template)
fig.show()

`Noida`, `Delhi` and `Pune` are the cities that have more than four thousand restaurants in our food delivery dataset.

In [103]:
fig = px.scatter(df1, x="Votes", y="Reviews", width=640, height=400, template=template)
fig.update_layout(title = "<b>Votes vs Reviews</b>")
fig.show()

From the above figure, we can infer that features `Reviews` and `Votes` are positively correlated in our dataset.

In [21]:
sns.set(rc={'figure.figsize':(10,8)})
sns.set_theme(style='whitegrid')

In [115]:
cityName="Pune"
title = "<b>Distribution of Reviews in "+cityName+ " for different locations</b>"
select_city = df1[(df1.city == cityName)]
fig = px.box(select_city, x='Reviews', y="Location", title=title, template=template, width=800, height=400)
fig.update_traces(marker_color=cityColor[cityName][1], marker_line_color=cityColor[cityName][0],
                  marker_line_width=2)
fig.show()

There are many outliers in the distribution of reviews for different locations in Pune city. however, the location `Pune University` has the highest median value of 58 reviews. The maximum number of reviews for a restaurant in Pune is around 3000 in our dataset.

In [42]:
cityName = "Gurgaon"
title="<b>Popular cuisines in " +cityName+ "</b>"
city_cuisine = df1[df1.city==cityName][['cuisine']]
top = city_cuisine.cuisine.value_counts().reset_index(level=0)
top.columns = ['Cuisine','Restaurants']
top = top[top['Restaurants']>30]

fig = px.bar(top, x='Cuisine', y='Restaurants', title=title, labels={'Restaurants':'Total Number of Restaurants'}, width=700, height=350, 
             template=template)
fig.update_traces(marker_color=cityColor[cityName][1], marker_line_color=cityColor[cityName][0],
                  marker_line_width=1.5, opacity=0.6)
fig.update_xaxes(tickangle=45)
fig.show()

In [209]:
cuisine_time = df1.groupby(['cuisine','minOrder'])['deliveryTime'].agg(['median','mean','min','max','count']).round()
cuisine_time.reset_index(level=['cuisine','minOrder'], inplace=True)
cuisine_time = cuisine_time[cuisine_time['count'] > 30]
cuisine_time['minOrder']=cuisine_time.minOrder.astype('int')
cuisine_time.columns = ['Cuisines','Minimum Order Value','Delivery Time Median','Average Delivery Time Taken','Minimum Delivery Time Taken','Maximum Delivery Time Taken','count']
cuisine_time.head()

,Cuisines,Minimum Order Value,Delivery Time Median,Average Delivery Time Taken,Minimum Delivery Time Taken,Maximum Delivery Time Taken,count
4,American,50,30.0,39.0,30,65,114
6,Andhra,50,45.0,43.0,30,65,38
8,Arabian,50,30.0,36.0,30,65,44
11,Asian,50,45.0,40.0,30,65,177
20,BBQ,50,30.0,38.0,30,65,34


Average delivery time taken by different cuisines

In [210]:
y='Average Delivery Time Taken'
fig = px.strip(cuisine_time, x='Cuisines', y=y,color='Minimum Order Value', template=template, width=1000, height=400, labels={y:y+' (in minutes)'})
fig.update_xaxes(tickangle=45)
fig.show()

The figure shows the average delivery time for popular cuisines for different restaurants. We can observe that most of the cuisines with a minimum order value of 50 rupees has delivery time within 30-40 minutes.

In [117]:
location_votes = df1.groupby(['city','Location'])['Votes','Reviews'].sum()
location_votes.reset_index(level=['city', 'Location'], inplace=True)
location_votes.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,city,Location,Votes,Reviews
0,Banglore,"BTM Layout 1, Electronic City",52948,19087
1,Banglore,"Mico Layout, Stage 2, BTM Layout,Bangalore",673232,326695
2,Delhi,"Babarpur, New Delhi, Delhi",29181,10104
3,Delhi,"Delhi Administration Flats, Timarpur",244669,124823
4,Delhi,Delhi Cantt.,136227,72522


In [120]:
fig = px.treemap(location_votes, path=[px.Constant('cities'),'city','Location'], values = "Votes", color='Reviews', labels={'Reviews':'Reviews'},
                 width=900,height=500, template=template)
fig.update_layout(title = "<b>Total number of Votes and Reviews for restaurants in different locations</b>")
fig.show()

The total number of votes are more than 6.4mn and with city Pune having the most number of votes in total for its restaurants. We have two locations in Delhi and Banglore having more than 300k reviews. 

In [128]:
df1.columns

Index(['Restaurant', 'Location', 'Votes', 'Reviews', 'deliveryTime',
       'numRating', 'avgCost', 'minOrder', 'city', 'cuisine'],
      dtype='object')

In [132]:
#top 10 most voted restaurants
top_restaurant = df1.groupby(['Restaurant','city'])['Votes'].sum()
top_restaurant.sort_values(ascending=False)[:10]

Restaurant  city   
ID_4539     Pune       101094
ID_2421     Pune        88695
ID_2041     Pune        77952
ID_13       Pune        46320
ID_5332     Delhi       42372
ID_2885     Pune        42219
ID_1666     Noida       39224
ID_5945     Delhi       39209
ID_7555     Kolkata     36996
ID_4360     Pune        35612
Name: Votes, dtype: int64

The most voted restaurant is in `Pune` city with more than 100k votes. From the top ten restaurants, six are in `Pune` and two are from `Delhi`.


Let's next answer the question, What is the average cost for popular cuisines in Mumbai vs the rest of the cities?

In [154]:
ind = (df1[df1['avgCost']=='']).index.to_list()
df1.drop(index=ind, inplace=True)
df1['avgCost'] = df1.avgCost.astype('int')

In [226]:
cityName = "Mumbai"

#popular cuisines in the selected city in sorted order
cuisine_cost_city = df1[df1.city==cityName].groupby(['cuisine'])['avgCost'].agg(['count', 'mean'])\
.reset_index(level=['cuisine']).sort_values(by=['count'], ascending=False)[:15]

#popular cuisines for the rest of the cities
cuisine_cost_rest = df1[df1.city!=cityName].groupby(['cuisine'])['avgCost'].agg(['count', 'mean'])

#xticks
x=cuisine_cost_city['cuisine'].values

#yticks for city
y1 = cuisine_cost_city['mean'].values

#yticks for the rest
y2 = []
for cuisine in x:
    #add the avg cost of the cuisine for rest of the cities
    y2.append(cuisine_cost_rest.loc[cuisine,'mean'])

In [227]:
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y1, 
                     name=cityName, 
                     marker_color=cityColor[cityName][0]))

fig.add_trace(go.Bar(x=x, y=y2, 
                     name='Rest of the cities', 
                     marker_color='rgb(66, 79, 94)'))

fig.update_layout(title="<b>Popular Cuisines in " +cityName+ "</b>", 
                  xaxis_tickfont_size=14,
                  xaxis=dict(title='Popular Cuisines',
                        titlefont_size=16,
                        tickfont_size=14),
                  yaxis=dict(title='Average Cost',
                        titlefont_size=16,
                        tickfont_size=14),
                  legend=dict(x=0,
                              y=1.0,
                              bgcolor='rgba(255, 255, 255, 0)',
                              bordercolor='rgba(255, 255, 255, 0)'),
                  barmode='group',
                  bargap=0.15, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.1, # gap between bars of the same location coordinate.
                  width=600,height=500

)
fig.show()

Excluding `Beverages`,`Ice Cream` and `Seafood`, all the other cuisines have slightly higher average cost than other cities in our data. The average cost of `Seafood` is lower than 50 rupees from the rest of the cities.

In [217]:
votes_reviews = df1.groupby('cuisine')[['Votes','Reviews']].sum().sort_values(by=['Votes'],ascending=False)
votes_reviews

,Votes,Reviews
cuisine,,
North Indian,925409,451194
Chinese,694761,342186
Fast Food,483055,226878
Italian,369321,208909
Continental,347305,196572
...,...,...
Bangladeshi,56,13
Hot dogs,37,13
Roast Chicken,18,10


In [250]:
votes_reviews = df1.groupby('cuisine')[['Votes','Reviews']].sum().sort_values(by=['Votes'],ascending=False)
x = votes_reviews.index.values[:10]
y1 = votes_reviews['Votes'].values[:10]
y2 = votes_reviews['Reviews'].values[:10]

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y1, 
                     name='Votes', 
                     marker_color='rgb(189, 245, 255)',
                     marker_line_color='rgb(9, 168, 189)',
                     marker_line_width=2))

fig.add_trace(go.Bar(x=x, y=y2, 
                     name='Reviews', 
                     marker_color='rgb(255, 194, 219)',
                     marker_line_color='rgb(158, 22, 78)',
                     marker_line_width=2))

fig.update_layout(title="<b>Votes and Reviews</b>", 
                  xaxis_tickfont_size=14,
                  xaxis=dict(title='Top Ten Cuisines',
                        titlefont_size=16,
                        tickfont_size=14),
                  yaxis=dict(title='Total Count',
                        titlefont_size=16,
                        tickfont_size=14),
                  legend=dict(x=0.75,
                              y=1.0,
                              bgcolor='rgba(255, 255, 255, 0)',
                              bordercolor='rgba(255, 255, 255, 0)'
                              ),
                  barmode='group',
                  bargap=0.4, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.1, # gap between bars of the same location coordinate.
                  width=600,height=500

)
fig.show()

What are the ratings for top 20 voted cuisines?

In [249]:
top_cuisine_rating = df1.groupby('cuisine')[['Votes','numRating']].agg(['sum','mean']).sort_values(by=[('Votes','sum')],ascending=False)[:20]
x= top_cuisine_rating.index.values
y = top_cuisine_rating[('numRating','mean')].values


fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y, name='Average Ratings', marker_color='rgb(255, 174, 0)', marker_line_color='rgb(250, 244, 187)', marker_line_width=2))
fig.update_layout(title="<b>Average Ratings</b>", 
                  xaxis_tickfont_size=14,
                  xaxis=dict(title='Top 20 Most Voted Cuisines',
                        titlefont_size=16,
                        tickfont_size=14,
                        tickangle=45),
                  yaxis=dict(title='Average Rating',
                        titlefont_size=16,
                        tickfont_size=14),
                  legend=dict(x=0.75,
                              y=1.0,
                              bgcolor='rgba(255, 255, 255, 0)',
                              bordercolor='rgba(255, 255, 255, 0)'
                              ),
                  bargap=0.4,
                  width=600,height=400)
fig.show()

Among the top twenty most voted cuisines, `Salad` and `Asian` cuisines have the highest average rating of 3.8 stars. All of the most voted cuisines have average ratings from 2.5 to 4 stars.